## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, max_error, mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor


## Preprocessing

In [2]:
#Read csv
df = pd.read_csv("../data/Data.csv")

In [3]:
#Change objects into correct datatypes
df['Target_Revenues']=df['Target_Revenues'].str.replace(',','')
df['Target_Revenues'] = df.Target_Revenues.astype(float)

df['Target_EBITDA']=df['Target_EBITDA'].str.replace(',','')
df['Target_EBITDA']=df.Target_EBITDA.astype(float)

df['Target_EV']=df['Target_EV'].str.replace(',','')
df['Target_EV']=df.Target_EV.astype(float)

df['Year']=df['Year'].str.replace(',','')
df['Year']=df.Year.astype(float)
df['Year']=df.Year.astype(int)
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')


In [4]:
#Remove "Government and Agencies" from Target Industry
df = df[df['Target_Industry_Macro'] != 'Government and Agencies']

In [5]:
#Remove nulls
df = df.dropna()

In [6]:
#Remove outliers
#Keep EV_Rev < 100
df = df[df['EV_Rev'] < 100]
#Keep EV_EBITDA >-100 and <100
df = df[df['EV_EBITDA'] > -100]
df = df[df['EV_EBITDA'] < 100]

In [7]:
df.shape

(13786, 17)

In [8]:
#Remove companies with EBITDA < Revenues
df = df[df['Target_Revenues'] > df['Target_EBITDA']]
df.shape

(13715, 17)

In [9]:
#One hot encoding

#Create country to region dictionary
country_to_region = {
    'United States': 'North America',
    'Canada': 'North America',
    'Greenland': 'North America',
    'France': 'Western Europe',
    'Germany': 'Western Europe',
    'Netherlands': 'Western Europe',
    'Belgium': 'Western Europe',
    'Luxembourg': 'Western Europe',
    'Switzerland': 'Western Europe',
    'Austria': 'Western Europe',
    'Liechtenstein': 'Western Europe',
    'Monaco': 'Western Europe',
    'United Kingdom': 'UK',
    'Jersey': 'UK',
    'Guernsey': 'UK',
    'Isle of Man': 'UK',
    'Ireland': 'UK',
    'Sweden': 'Nordics',
    'Norway': 'Nordics',
    'Denmark': 'Nordics',
    'Finland': 'Nordics',
    'Iceland': 'Nordics',
    'Poland': 'Eastern Europe',
    'Bosnia and Herzegovina': 'Eastern Europe',
    'Lithuania': 'Eastern Europe',
    'Bulgaria': 'Eastern Europe',
    'Russia': 'Eastern Europe',
    'Estonia': 'Eastern Europe',
    'Latvia': 'Eastern Europe',
    'Hungary': 'Eastern Europe',
    'Romania': 'Eastern Europe',
    'Ukraine': 'Eastern Europe',
    'Moldova': 'Eastern Europe',
    'Serbia': 'Eastern Europe',
    'Slovenia': 'Eastern Europe',
    'North Macedonia': 'Eastern Europe',
    'Montenegro': 'Eastern Europe',
    'Slovakia': 'Eastern Europe',
    'Czech Republic': 'Eastern Europe',
    'Portugal': 'Southern Europe',
    'Italy': 'Southern Europe',
    'Spain': 'Southern Europe',
    'Greece': 'Southern Europe',
    'Croatia': 'Southern Europe',
    'Cyprus': 'Southern Europe',
    'Malta': 'Southern Europe',
    'Gibraltar': 'Southern Europe',
    'Turkey': 'Southern Europe'
}

#Apply the updated mapping to create a new column
df['Target_Region'] = df['Target_Nation'].map(country_to_region)

#One-hot encode the 'Target_Region' column
df = pd.get_dummies(df, columns=['Target_Region'])
df = pd.get_dummies(df, columns=['Target_Industry_Macro'])
df = pd.get_dummies(df, columns=['Target_Status'])
df = df.drop(columns=['Target_Status_Private'])

In [10]:
#Log transformations
df['log_Target_EV'] = df['Target_EV'].apply(lambda x: np.log(x))
df['log_Target_Revenues'] = df['Target_Revenues'].apply(lambda x: np.log(x))

#neglog transformation to allow negative EBITDAs
df['log_Target_EBITDA'] = df['Target_EBITDA'].apply(lambda x: -np.log(abs(x)) if x < 0 else np.log(x))
#remove one row with value of -inf
df = df[df['log_Target_EBITDA'] != -np.inf]

In [11]:
#Engineer "Market_Condition" feature
# Step 1: Create 'YearMonth' from the 'Date' column for grouping and calculations
df['YearMonth'] = df['Date'].dt.to_period('M')

# Step 2: Calculate the monthly median EV/EBITDA for each YearMonth
monthly_median = df.groupby('YearMonth')['EV_EBITDA'].median().reset_index()

# Step 3: Rename the median column for clarity
monthly_median.rename(columns={'EV_EBITDA': 'EV_EBITDA_MonthlyMedian'}, inplace=True)

# Step 4: Drop 'EV_EBITDA' column before merging to avoid duplicates
df.drop('EV_EBITDA', axis=1, inplace=True)

# Step 5: Merge the monthly medians back into the original DataFrame
df = pd.merge(df, monthly_median, on='YearMonth', how='left')

# Step 6: Calculate quantiles for the monthly medians
quantiles = df['EV_EBITDA_MonthlyMedian'].quantile([0.1, 0.5, 0.9])
q1 = quantiles[0.1]  # First quantile
q2 = quantiles[0.5]   # Median
q3 = quantiles[0.9]  # Third quantile

# Step 7: Define function to classify market conditions
def classify_market_condition(row):
    if row['EV_EBITDA_MonthlyMedian'] < q1:
        return 'LM'  # Low Market
    elif row['EV_EBITDA_MonthlyMedian'] <= q3:
        return 'NM'  # Normal Market
    else:
        return 'HM'  # High Market

# Step 8: Apply the function to create the ‘Market_Condition’ column
df['Market_Condition'] = df.apply(classify_market_condition, axis=1)

# Step 9: One-hot encode the ‘Market_Condition’ column
df['Market_Condition_A'] = df['Market_Condition']
df = pd.get_dummies(df, columns=['Market_Condition'], prefix='Market')

In [12]:
# Display the first few rows of the DataFrame to ensure everything worked
df.head()

,Date,Target_Name,Target_Industry_Mid,Target_Nation,Buyer_Name,Buyer_Industry_Macro,Buyer_Industry_Mid,Buyer_Nation,Target_Revenues,Target_EBITDA,...,Target_Status_Public,log_Target_EV,log_Target_Revenues,log_Target_EBITDA,YearMonth,EV_EBITDA_MonthlyMedian,Market_Condition_A,Market_HM,Market_LM,Market_NM
0,2024-08-14,Sarsys-Asft AB,Other Industrials,Sweden,Grundbulten 137100 AB,Financials,Other Financials,Sweden,3.76,-0.23,...,1,1.057790,1.324419,1.469676,2024-08,2.35,LM,0,1,0
1,2024-08-08,GSE Systems Inc,Software,United States,Nuclear Engineering Holdings LLC,Financials,Other Financials,United States,41.81,-1.02,...,1,2.577942,3.733136,-0.019803,2024-08,2.35,LM,0,1,0
2,2024-08-06,INEO Tech Corp,Professional Services,Canada,Coenda Investment Holdings Corp,Financials,Other Financials,Canada,0.98,-1.78,...,1,1.363537,-0.020203,-0.576613,2024-08,2.35,LM,0,1,0
3,2024-08-03,Big Cheese Studio SA,Software,Poland,Investor Group,Financials,Other Financials,Poland,2.71,1.05,...,1,2.423917,0.996949,0.048790,2024-08,2.35,LM,0,1,0
4,2024-08-01,Braille Energy Systems Inc,Other Energy & Power,Canada,Undisclosed Acquiror,Financials,Brokerage,Unknown,2.43,-1.35,...,1,1.768150,0.887891,-0.300105,2024-08,2.35,LM,0,1,0


## Best model - Random Forest

In [15]:
# Step 1: Prepare the Data
X = df[['log_Target_Revenues', 'log_Target_EBITDA', 'Market_HM', 'Market_NM', 'Market_LM', \
        'Target_Status_Public', 'Target_Region_Eastern Europe', 'Target_Region_Nordics', \
        'Target_Region_North America', 'Target_Region_Southern Europe', 'Target_Region_UK', \
        'Target_Region_Western Europe', 'Target_Industry_Macro_Consumer Products and Services', \
        'Target_Industry_Macro_Consumer Staples','Target_Industry_Macro_Energy and Power', \
        'Target_Industry_Macro_Financials','Target_Industry_Macro_Healthcare', \
        'Target_Industry_Macro_High Technology','Target_Industry_Macro_Industrials', \
        'Target_Industry_Macro_Materials','Target_Industry_Macro_Media and Entertainment', \
        'Target_Industry_Macro_Real Estate','Target_Industry_Macro_Retail', \
        'Target_Industry_Macro_Telecommunications']]
y = df['log_Target_EV']

# Step 2: Split the Data into Training and Testing Sets
strata = pd.qcut(df.log_Target_EV, 10, labels=False) + 1
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=strata, test_size=0.3, random_state=1234)

# Step 3: Standardize the Data (important for Random Forest in some cases, though not strictly necessary)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 4: Define the parameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': range(10, 500, 10),  # Number of trees in the forest
    'max_depth': [None] + list(range(10, 101, 10)),  # Depth of each tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required at each leaf node
    'max_features': np.linspace(.1, .99, num=100) # Number of features to consider when looking for the best split
}

# Step 5: Perform Randomized Search with cross-validation
random_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42),  # Initialize the RandomForestRegressor
    param_distributions=param_dist,
    n_iter=50,  # Number of random samples to evaluate (increase if needed)
    cv=5,  # 5-fold cross-validation
    scoring='neg_mean_squared_error',  # Use negative MSE as the scoring metric
    n_jobs=-1,  # Use all available cores
    verbose=1,  # Display progress
    random_state=42  # For reproducibility
)

# Step 6: Fit RandomizedSearchCV to the scaled training data
random_search.fit(X_train_scaled, y_train)

# Step 7: Get the best parameters and model
best_rf = random_search.best_estimator_
print(f'Best parameters from RandomizedSearchCV: {random_search.best_params_}')

# Step 8: Predict on Training and Testing Sets with the best model
y_train_pred = best_rf.predict(X_train_scaled)
y_test_pred = best_rf.predict(X_test_scaled)

# Unlog
y_train = np.exp(y_train)
y_train_pred = np.exp(y_train_pred)
y_test = np.exp(y_test)
y_test_pred = np.exp(y_test_pred)

# Step 6: Evaluate the Model
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)
train_max_error = max_error(y_train, y_train_pred)
train_mape = mean_absolute_percentage_error(y_train, y_train_pred)
test_max_error = max_error(y_test, y_test_pred)
test_mape = mean_absolute_percentage_error(y_test, y_test_pred)

# Print the evaluation metrics
print('Performance on Training Data:')
print(f'Root Mean Squared Error (Train): {train_rmse}')
print(f'R^2 Score (Train): {train_r2}')
print(f'Max Error (Train): {train_max_error}')
print(f'MAPE (Train): {train_mape}')

print('\nPerformance on Testing Data:')
print(f'Root Mean Squared Error (Test): {test_rmse}')
print(f'R^2 Score (Test): {test_r2}')
print(f'Max Error (Test): {test_max_error}')
print(f'MAPE (Test): {test_mape}')

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters from RandomizedSearchCV: {'n_estimators': 440, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 0.46858585858585866, 'max_depth': 70}
Performance on Training Data:
Root Mean Squared Error (Train): 22918.408246646435
R^2 Score (Train): 0.6253531644891837
Max Error (Train): 1943761.3575665432
MAPE (Train): 0.6239172282375631

Performance on Testing Data:
Root Mean Squared Error (Test): 33572.70735217626
R^2 Score (Test): 0.5341152112912606
Max Error (Test): 1947723.2875665405
MAPE (Test): 1.3323163693721418
